<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/asosMetar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## From https://mesonet.agron.iastate.edu/request/download.phtml?network=CO_ASOS
## https://github.com/akrherz/iem/blob/main/scripts/asos/iem_scraper_example.py

In [51]:
import json
import time
import datetime
from urllib.request import urlopen
import pandas as pd

In [8]:
# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"

In [9]:
def download_data(uri):
    """Fetch the data from the IEM
    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.
    Args:
      uri (string): URL to fetch
    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode("utf-8")
            if data is not None and not data.startswith("ERROR"):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""

In [10]:
def get_stations_from_filelist(filename):
    """Build a listing of stations from a simple file listing the stations.
    The file should simply have one station per line.
    """
    stations = []
    for line in open(filename):
        stations.append(line.strip())
    return 

In [33]:
def get_stations_from_networks():
    """Build a station list by using a bunch of IEM networks."""
    stations = []
    #states = """AK AL AR AZ CA CO CT DE FL GA HI IA ID IL IN KS KY LA MA MD ME
    #MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT
    #WA WI WV WY"""
    states = """MD DC""" # my states option
    networks = []
    for state in states.split():
        networks.append("%s_ASOS" % (state,))

    for network in networks:
        # Get metadata
        uri = (
            "https://mesonet.agron.iastate.edu/geojson/network/%s.geojson"
        ) % (network,)
        data = urlopen(uri)
        jdict = json.load(data)
        for site in jdict["features"]:
            stations.append(site["properties"]["sid"])
    return stations

In [34]:
def download_alldata():
    """An alternative method that fetches all available data.
    Service supports up to 24 hours worth of data at a time."""
    # timestamps in UTC to request data for
    startts = datetime.datetime(2012, 8, 1)
    endts = datetime.datetime(2012, 9, 1)
    interval = datetime.timedelta(hours=24)

    service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"

    now = startts
    while now < endts:
        thisurl = service
        thisurl += now.strftime("year1=%Y&month1=%m&day1=%d&")
        thisurl += (now + interval).strftime("year2=%Y&month2=%m&day2=%d&")
        print("Downloading: %s" % (now,))
        data = download_data(thisurl)
        outfn = "%s.txt" % (now.strftime("%Y%m%d"),)
        with open(outfn, "w") as fh:
            fh.write(data)
        now += interval

In [35]:
# timestamps in UTC to request data for
startts = datetime.datetime(2022, 5, 1)
endts = datetime.datetime(2022, 5, 9)

In [36]:
service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"

service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
service += endts.strftime("year2=%Y&month2=%m&day2=%d&")

# Two examples of how to specify a list of stations
stations = get_stations_from_networks()

In [38]:
service

'http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2022&month1=05&day1=01&year2=2022&month2=05&day2=09&'

In [20]:
#stations = ['KBWI']

In [37]:
stations

['ADW',
 'NAK',
 'DMH',
 'MTN',
 'BWI',
 'CGE',
 'RSP',
 'CGS',
 'CBE',
 'ESN',
 'FME',
 'FDK',
 'GAI',
 'HGR',
 '2W6',
 '2G4',
 'OXB',
 'NHK',
 'APG',
 'RJD',
 'SBY',
 'W29',
 'NUI',
 'DMW']

In [39]:
    for station in stations:
        uri = "%s&station=%s" % (service, station)
        print("Downloading: %s" % (station,))
        data = download_data(uri)
        outfn = "%s_%s_%s.txt" % (
            station,
            startts.strftime("%Y%m%d%H%M"),
            endts.strftime("%Y%m%d%H%M"),
        )
        out = open(outfn, "w")
        out.write(data)
        out.close()

Downloading: ADW
Downloading: NAK
Downloading: DMH
Downloading: MTN
Downloading: BWI
Downloading: CGE
Downloading: RSP
Downloading: CGS
Downloading: CBE
Downloading: ESN
Downloading: FME
Downloading: FDK
Downloading: GAI
Downloading: HGR
Downloading: 2W6
Downloading: 2G4
Downloading: OXB
Downloading: NHK
Downloading: APG
Downloading: RJD
Downloading: SBY
Downloading: W29
Downloading: NUI
Downloading: DMW


In [40]:
data

'#DEBUG: Format Typ    -> comma\n#DEBUG: Time Period   -> 2022-05-01 00:00:00+00:00 2022-05-09 00:00:00+00:00\n#DEBUG: Time Zone     -> Etc/UTC\n#DEBUG: Data Contact   -> daryl herzmann akrherz@iastate.edu 515-294-5978\n#DEBUG: Entries Found -> -1\nstation,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth\nDMW,2022-05-01 00:15,-77.0077,39.6083,62.60,30.20,29.35,0.00,0.00,0.00,30.09,M,10.00,M,CLR,M,M,M,M,M,M,M,M,M,M,M,M,M,M,62.60,KDMW 010015Z AUTO 00000KT 10SM CLR 17/M01 A3009 RMK AO2,M\nDMW,2022-05-01 00:35,-77.0077,39.6083,60.80,30.20,31.28,0.00,0.00,0.00,30.10,M,10.00,M,CLR,M,M,M,M,M,M,M,M,M,M,M,M,M,M,60.80,KDMW 010035Z AUTO 00000KT 10SM CLR 16/M01 A3010 RMK AO2 LTG DSNT W,M\nDMW,2022-05-01 00:55,-77.0077,39.6083,59.00,32.00,35.87,0.00,0.00,0.00,30.10,M,10.00,M,CLR,M,M,M,M,M,M,M,M,M,M,M,M,M,M,59.0

In [41]:
outfn

'DMW_202205010000_202205090000.txt'

In [42]:
out

<_io.TextIOWrapper name='DMW_202205010000_202205090000.txt' mode='w' encoding='UTF-8'>

In [49]:
f2read = 'NUI_202205010000_202205090000.txt'

In [50]:
with open(f2read, "r") as file1:
    FileContent = file1.read()
    print(FileContent)

#DEBUG: Format Typ    -> comma
#DEBUG: Time Period   -> 2022-05-01 00:00:00+00:00 2022-05-09 00:00:00+00:00
#DEBUG: Time Zone     -> Etc/UTC
#DEBUG: Data Contact   -> daryl herzmann akrherz@iastate.edu 515-294-5978
#DEBUG: Entries Found -> -1
station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
NUI,2022-05-01 00:53,-76.4285,38.1460,55.00,30.90,39.63,130.00,5.00,0.00,30.11,1019.60,10.00,M,CLR,M,M,M,M,M,M,M,M,M,M,M,M,M,M,55.00,KNUI 010053Z AUTO 13005KT 10SM CLR 13/M01 A3011 RMK AO2 SLP196 T01281006 $,M
NUI,2022-05-01 01:53,-76.4285,38.1460,55.00,37.00,50.58,140.00,4.00,0.00,30.11,1019.70,10.00,M,CLR,M,M,M,M,M,M,M,M,M,M,M,M,M,M,55.00,KNUI 010153Z AUTO 14004KT 10SM CLR 13/03 A3011 RMK AO2 SLP197 T01280028 $,M
NUI,2022-05-01 02:53,-76.4285,38.1460,52.00,41.00,66.00,150.00,4.00,0.00,30.11,1019.40,10.

In [ ]:
testData = pd.read_csv('NUI_202205010000_202205090000.txt', skiprows=5 )

In [ ]:
testData.head(2)

In [ ]:
testData.info()

In [61]:
testData.T

,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,286,287
station,NUI,NUI,NUI,NUI,NUI,NUI,NUI,NUI,NUI,NUI,...,NUI,NUI,NUI,NUI,NUI,NUI,NUI,NUI,NUI,NUI
valid,2022-05-01 00:53,2022-05-01 01:53,2022-05-01 02:53,2022-05-01 03:53,2022-05-01 04:53,2022-05-01 05:53,2022-05-01 06:53,2022-05-01 07:53,2022-05-01 08:53,2022-05-01 09:53,...,2022-05-08 15:53,2022-05-08 16:53,2022-05-08 17:53,2022-05-08 18:53,2022-05-08 19:26,2022-05-08 19:53,2022-05-08 20:53,2022-05-08 21:53,2022-05-08 22:53,2022-05-08 23:53
lon,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,...,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285,-76.4285
lat,38.146,38.146,38.146,38.146,38.146,38.146,38.146,38.146,38.146,38.146,...,38.146,38.146,38.146,38.146,38.146,38.146,38.146,38.146,38.146,38.146
tmpf,55.0,55.0,52.0,57.0,57.0,55.9,54.0,53.1,53.1,51.1,...,51.1,52.0,53.1,53.1,54.0,54.0,55.0,55.0,54.0,54.0
dwpf,30.90,37.00,41.00,46.90,48.00,46.90,46.00,46.90,46.00,46.00,...,43.00,42.10,42.10,42.10,42.10,41.00,39.90,39.90,37.00,37.00
relh,39.63,50.58,66.00,68.90,71.81,71.69,74.26,79.40,76.75,82.62,...,73.69,68.86,66.13,66.13,63.98,61.32,56.66,56.66,52.45,52.45
drct,130.00,140.00,150.00,180.00,190.00,190.00,0.00,0.00,190.00,0.00,...,30.00,30.00,30.00,M,30.00,40.00,M,40.00,50.00,M
sknt,5.0,4.0,4.0,6.0,4.0,3.0,0.0,0.0,4.0,0.0,...,8.0,9.0,8.0,6.0,7.0,8.0,6.0,9.0,9.0,6.0
p01i,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
